In [1]:
# importing relevant packages 

import torch 
from torchvision import models
import pandas as pd
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn 
import glob
import os
import cv2
import matplotlib.pyplot as plt 
from matplotlib import pyplot
import torchvision.transforms as transforms
from torchsummary import summary
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
import json
from PIL import Image
import random
import albumentations as A
import glob
from shapely.geometry import Polygon

/home/dell/Desktop/xxxx/Uni/Kirigami_project/Keypoint_detection_notebooks/.Keypoint_detection_notebooks/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
class VGG_model(nn.Module):
    def __init__(self):
        super(VGG_model, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        )
        
        self.avgpool = nn.Sequential(
            nn.Conv2d(512,512, kernel_size=3, padding='same'),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(512,50, kernel_size=3, padding='same'),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.AdaptiveAvgPool2d(output_size=(8,8))
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(3200, 300),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Dropout(0.3),
            nn.Linear(300, 32),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
        

In [3]:
model = VGG_model()
model.load_state_dict(torch.load('./Model_VGG_7_1000_22100_epoch'))
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)
model.eval()

VGG_model(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.1, inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): LeakyReLU(negative_slope=0.1, inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): LeakyReLU(negative_slope=0.1, inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): LeakyReLU(negative_slope=0.1, inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): LeakyReLU(negative_slope=0.1, inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): LeakyReLU(negative_slope=0.1, inplace=True)
    (14): Conv2d(

In [4]:
def predict_keypoints(img):
    img = cv2.resize(img, (224,224))
    img_tensor = transforms.ToTensor()(img)
    img_mean = img_tensor.mean(dim = (1,2))
    img_std = img_tensor.std(dim = (1,2))
        
    img_normalised = transforms.Normalize(img_mean, img_std)(img_tensor)
    img_normalised = img_normalised.to(device)
    
    key_points = model(img_normalised[None]).flatten().detach().cpu().numpy()
    
    return key_points

In [5]:
def plot_keypoints(img, keypoints):                                                             

    plt.imshow(img)

    x_points = keypoints[0::2]
    y_points = keypoints[1::2]
    plt.scatter(x_points*img.shape[1], y_points*img.shape[0], s = 4, c=(1,0,0))
    plt.show()

In [6]:
def process_num(x):
    return x*random.uniform(0.96, 1.04)

In [43]:
dataset_names = ['Grasp_dataset_cylinder_20', 'Grasp_dataset_cylinder_40', 'Grasp_dataset_cylinder_60', 'Grasp_dataset_square_20', 'Grasp_dataset_square_40', 'Grasp_dataset_square_60']
# dataset_names = ['Grasp_dataset_cylinder_20', 'Grasp_dataset_cylinder_40', 'Grasp_dataset_cylinder_60']

header = ['weight_reading_1', 'weight_reading_2', 'pressure_reading_1', 'pressure_reading_2', 'force_reading_1', 'force_reading_2', 'p1_x', 'p1_y', 'p2_x', 'p2_y', 'p3_x', 'p3_y', 'p4_x', 'p4_y', 'p5_x', 'p5_y', 'p6_x', 'p6_y', 'p7_x', 'p7_y', 'p8_x', 'p8_y', 'p9_x', 'p9_y', 'p10_x', 'p10_y', 'p11_x', 'p11_y', 'p12_x', 'p12_y', 'p13_x', 'p13_y', 'p14_x', 'p14_y', 'p15_x', 'p15_y', 'p16_x', 'p16_y', 'label']

current_dir = os. getcwd()

weight_reading_1_arr = []
weight_reading_2_arr = []
pressure_reading_1_arr = []
pressure_reading_2_arr = []
force_reading_1_arr = []
force_reading_2_arr = []

p1_x_arr = []
p1_y_arr = []
p2_x_arr = []
p2_y_arr = []
p3_x_arr = []
p3_y_arr = []
p4_x_arr = []
p4_y_arr = []
p5_x_arr = []
p5_y_arr = []
p6_x_arr = []
p6_y_arr = []
p7_x_arr = []
p7_y_arr = []
p8_x_arr = []
p8_y_arr = []
p9_x_arr = []
p9_y_arr = []
p10_x_arr = []
p10_y_arr = []
p11_x_arr = []
p11_y_arr = []
p12_x_arr = []
p12_y_arr = []
p13_x_arr = []
p13_y_arr = []
p14_x_arr = []
p14_y_arr = []
p15_x_arr = []
p15_y_arr = []
p16_x_arr = []
p16_y_arr = []
label_arr = []


for f in dataset_names:
    
#     image_dir = os.path.join(current_dir + '/Grasp_dataset_7', f)
    image_dir = os.path.join(current_dir + '/Grasp_dataset_7_test', f)
    data_dir = os.path.join(image_dir, f + '.csv')
    df = pd.read_csv(data_dir)
    weight_reading_1_arr = np.concatenate((weight_reading_1_arr, df['weight_reading_1'].values))
    weight_reading_2_arr = np.concatenate((weight_reading_2_arr, df['weight_reading_2'].values))
    pressure_reading_1_arr = np.concatenate((pressure_reading_1_arr, df['pressure_reading_1'].values))
    pressure_reading_2_arr = np.concatenate((pressure_reading_2_arr, df['pressure_reading_2'].values))
    force_reading_1_arr = np.concatenate((force_reading_1_arr, df['force_reading_1'].values))
    force_reading_2_arr = np.concatenate((force_reading_2_arr, df['force_reading_2'].values))
    label_arr = np.concatenate((label_arr, df['label'].values))
    
    j = 0
    
    while j < df.shape[0]:
        image = cv2.imread(os.path.join(image_dir, df.iloc[j]['image_name']))
        keypoints = predict_keypoints(image)
        
#         plot_keypoints(image, keypoints)
        
#         if j == 20:
#             plot_keypoints(image, keypoints)
        

#         plt.imshow(image)

#         plt.scatter(kirigami_x_points*image.shape[1], kirigami_y_points*image.shape[0], s = 4, c=(1,0,0))
#         plt.show()
        
        p1_x_arr.append(keypoints[0])
        p1_y_arr.append(keypoints[1])
        p2_x_arr.append(keypoints[2])
        p2_y_arr.append(keypoints[3])
        p3_x_arr.append(keypoints[4])
        p3_y_arr.append(keypoints[5])
        p4_x_arr.append(keypoints[6])
        p4_y_arr.append(keypoints[7])
        p5_x_arr.append(keypoints[8])
        p5_y_arr.append(keypoints[9])
        p6_x_arr.append(keypoints[10])
        p6_y_arr.append(keypoints[11])
        p7_x_arr.append(keypoints[12])
        p7_y_arr.append(keypoints[13])
        p8_x_arr.append(keypoints[14])
        p8_y_arr.append(keypoints[15])
        p9_x_arr.append(keypoints[16])
        p9_y_arr.append(keypoints[17])
        p10_x_arr.append(keypoints[18])
        p10_y_arr.append(keypoints[19])
        p11_x_arr.append(keypoints[20])
        p11_y_arr.append(keypoints[21])
        p12_x_arr.append(keypoints[22])
        p12_y_arr.append(keypoints[23])
        p13_x_arr.append(keypoints[24])
        p13_y_arr.append(keypoints[25])
        p14_x_arr.append(keypoints[26])
        p14_y_arr.append(keypoints[27])
        p15_x_arr.append(keypoints[28])
        p15_y_arr.append(keypoints[29])
        p16_x_arr.append(keypoints[30])
        p16_y_arr.append(keypoints[31])
        
        j = j + 1
        
#         if j == 2:
#             break
#     break



p1_x_arr = np.array(p1_x_arr) - np.array(p1_x_arr)
p2_x_arr = np.array(p2_x_arr) - np.array(p1_x_arr)
p3_x_arr = np.array(p3_x_arr) - np.array(p1_x_arr)
p4_x_arr = np.array(p4_x_arr) - np.array(p1_x_arr)
p5_x_arr = np.array(p5_x_arr) - np.array(p1_x_arr)
p6_x_arr = np.array(p6_x_arr) - np.array(p1_x_arr)
p7_x_arr = np.array(p7_x_arr) - np.array(p1_x_arr)
p8_x_arr = np.array(p8_x_arr) - np.array(p1_x_arr)
p9_x_arr = np.array(p9_x_arr) - np.array(p1_x_arr)
p10_x_arr = np.array(p10_x_arr) - np.array(p1_x_arr)
p11_x_arr = np.array(p11_x_arr) - np.array(p1_x_arr)
p12_x_arr = np.array(p12_x_arr) - np.array(p1_x_arr)
p13_x_arr = np.array(p13_x_arr) - np.array(p1_x_arr)
p14_x_arr = np.array(p14_x_arr) - np.array(p1_x_arr)
p15_x_arr = np.array(p15_x_arr) - np.array(p1_x_arr)
p16_x_arr = np.array(p16_x_arr) - np.array(p1_x_arr)

p1_y_arr = np.array(p1_y_arr) - np.array(p1_y_arr)
p2_y_arr = np.array(p2_y_arr) - np.array(p1_y_arr)
p3_y_arr = np.array(p3_y_arr) - np.array(p1_y_arr)
p4_y_arr = np.array(p4_y_arr) - np.array(p1_y_arr)
p5_y_arr = np.array(p5_y_arr) - np.array(p1_y_arr)
p6_y_arr = np.array(p6_y_arr) - np.array(p1_y_arr)
p7_y_arr = np.array(p7_y_arr) - np.array(p1_y_arr)
p8_y_arr = np.array(p8_y_arr) - np.array(p1_y_arr)
p9_y_arr = np.array(p9_y_arr) - np.array(p1_y_arr)
p10_y_arr = np.array(p10_y_arr) - np.array(p1_y_arr)
p11_y_arr = np.array(p11_y_arr) - np.array(p1_y_arr)
p12_y_arr = np.array(p12_y_arr) - np.array(p1_y_arr)
p13_y_arr = np.array(p13_y_arr) - np.array(p1_y_arr)
p14_y_arr = np.array(p14_y_arr) - np.array(p1_y_arr)
p15_y_arr = np.array(p15_y_arr) - np.array(p1_y_arr)
p16_y_arr = np.array(p16_y_arr) - np.array(p1_y_arr)


weight_diff_1_arr = np.array(weight_reading_1_arr) - np.array(force_reading_1_arr)
weight_diff_2_arr = np.array(force_reading_2_arr) - np.array(weight_reading_2_arr) 

grasp_dataset = pd.concat([pd.Series(weight_diff_1_arr), pd.Series(weight_diff_2_arr), pd.Series(pressure_reading_1_arr), pd.Series(pressure_reading_2_arr), pd.Series(force_reading_1_arr), pd.Series(force_reading_2_arr), pd.Series(p1_x_arr), pd.Series(p1_y_arr), pd.Series(p2_x_arr), pd.Series(p2_y_arr), pd.Series(p3_x_arr), pd.Series(p3_y_arr), pd.Series(p4_x_arr), pd.Series(p4_y_arr), pd.Series(p5_x_arr), pd.Series(p5_y_arr), pd.Series(p6_x_arr), pd.Series(p6_y_arr), pd.Series(p7_x_arr), pd.Series(p7_y_arr), pd.Series(p8_x_arr), pd.Series(p8_y_arr), pd.Series(p9_x_arr), pd.Series(p9_y_arr), pd.Series(p10_x_arr), pd.Series(p10_y_arr), pd.Series(p11_x_arr), pd.Series(p11_y_arr), pd.Series(p12_x_arr), pd.Series(p12_y_arr), pd.Series(p13_x_arr), pd.Series(p13_y_arr), pd.Series(p14_x_arr), pd.Series(p14_y_arr), pd.Series(p15_x_arr), pd.Series(p15_y_arr), pd.Series(p16_x_arr), pd.Series(p16_y_arr), pd.Series(label_arr)], axis=1, keys=header)

# num_cols = grasp_dataset.select_dtypes(include=['float']).columns
# df[num_cols] = grasp_dataset[num_cols].applymap(process_num)

# combined_df_set = grasp_dataset
# combined_df_set = pd.concat([combined_df_set, grasp_dataset], axis=0)


In [44]:
pd.options.display.max_columns = None
print(grasp_dataset.head(3))

   weight_reading_1  weight_reading_2  pressure_reading_1  pressure_reading_2  \
0              72.0              61.6                39.4                57.3   
1              74.8              59.3                41.7                64.4   
2              73.4              60.8                43.5                65.3   

   force_reading_1  force_reading_2  p1_x  p1_y      p2_x      p2_y      p3_x  \
0            264.3            263.7   0.0   0.0  0.474616  0.177147  0.800419   
1            270.0            269.3   0.0   0.0  0.476011  0.172187  0.800696   
2            278.5            277.8   0.0   0.0  0.477857  0.168484  0.801501   

       p3_y      p4_x      p4_y      p5_x      p5_y      p6_x      p6_y  \
0  0.341526  0.798435  0.722489  0.479104  0.872773  0.184380  0.701780   
1  0.348850  0.797162  0.716827  0.478588  0.875796  0.182801  0.702902   
2  0.358086  0.797434  0.709199  0.479064  0.879742  0.179297  0.704517   

       p7_x      p7_y      p8_x      p8_y      p9

In [45]:
grasp_dataset

,weight_reading_1,weight_reading_2,pressure_reading_1,pressure_reading_2,force_reading_1,force_reading_2,p1_x,p1_y,p2_x,p2_y,p3_x,p3_y,p4_x,p4_y,p5_x,p5_y,p6_x,p6_y,p7_x,p7_y,p8_x,p8_y,p9_x,p9_y,p10_x,p10_y,p11_x,p11_y,p12_x,p12_y,p13_x,p13_y,p14_x,p14_y,p15_x,p15_y,p16_x,p16_y,label
0,72.0,61.6,39.4,57.3,264.3,263.7,0.0,0.0,0.474616,0.177147,0.800419,0.341526,0.798435,0.722489,0.479104,0.872773,0.184380,0.701780,0.466461,0.263490,0.463288,0.391953,0.464998,0.531428,0.470341,0.662470,0.474164,0.784578,0.498773,0.783616,0.509421,0.658661,0.513727,0.528584,0.507315,0.403731,0.497109,0.271870,cylinder_20
1,74.8,59.3,41.7,64.4,270.0,269.3,0.0,0.0,0.476011,0.172187,0.800696,0.348850,0.797162,0.716827,0.478588,0.875796,0.182801,0.702902,0.466307,0.263670,0.461439,0.392506,0.462530,0.531179,0.468002,0.661252,0.472812,0.783449,0.497517,0.782665,0.509018,0.657450,0.513974,0.528182,0.507635,0.404358,0.497309,0.272195,cylinder_20
2,73.4,60.8,43.5,65.3,278.5,277.8,0.0,0.0,0.477857,0.168484,0.801501,0.358086,0.797434,0.709199,0.479064,0.879742,0.179297,0.704517,0.465896,0.265107,0.459122,0.394094,0.459172,0.531317,0.464990,0.660130,0.471330,0.782750,0.496903,0.781894,0.508882,0.656457,0.514250,0.527892,0.507887,0.405053,0.497503,0.273041,cylinder_20
3,71.1,63.2,42.7,64.6,281.9,281.0,0.0,0.0,0.479748,0.167117,0.800319,0.364467,0.796531,0.703486,0.480494,0.882557,0.178299,0.707288,0.465591,0.265188,0.457853,0.394158,0.457528,0.530779,0.463566,0.659180,0.470623,0.782402,0.496909,0.782135,0.508299,0.656414,0.513394,0.527662,0.507145,0.404596,0.497500,0.272908,cylinder_20
4,70.5,63.4,42.6,64.8,281.1,279.8,0.0,0.0,0.480382,0.168413,0.799666,0.368086,0.796786,0.701430,0.480778,0.881237,0.176580,0.708442,0.465257,0.266333,0.456843,0.395022,0.456182,0.531077,0.462328,0.659072,0.470153,0.782219,0.496576,0.782354,0.507900,0.656943,0.512860,0.528126,0.506559,0.405259,0.497090,0.274019,cylinder_20
5,68.0,65.9,40.1,64.3,275.3,274.0,0.0,0.0,0.481535,0.171460,0.797417,0.369723,0.794492,0.693392,0.482313,0.878144,0.176351,0.715249,0.464910,0.265700,0.456576,0.394066,0.456065,0.530506,0.461949,0.659322,0.469398,0.782308,0.495959,0.782653,0.505182,0.657577,0.509442,0.527720,0.503726,0.403673,0.496415,0.273034,cylinder_20
6,72.1,62.0,43.7,65.4,280.5,279.4,0.0,0.0,0.479288,0.167450,0.800932,0.362867,0.797125,0.704389,0.480613,0.881430,0.178524,0.706989,0.465641,0.265280,0.458157,0.394113,0.458029,0.530740,0.464081,0.659142,0.470965,0.782053,0.497145,0.781780,0.508402,0.656252,0.513424,0.527670,0.507144,0.404676,0.497497,0.273114,cylinder_20
7,70.8,63.4,43.5,64.4,280.6,279.4,0.0,0.0,0.480764,0.167590,0.800019,0.367134,0.796716,0.701114,0.480766,0.880674,0.176989,0.707891,0.465638,0.265600,0.457169,0.394429,0.456534,0.530623,0.462621,0.658710,0.470250,0.781769,0.496636,0.781949,0.507974,0.656564,0.513021,0.527724,0.506862,0.404766,0.497568,0.273393,cylinder_20
8,74.6,59.2,41.8,65.6,271.4,270.3,0.0,0.0,0.476694,0.170789,0.801311,0.350694,0.797406,0.714993,0.478676,0.877393,0.181664,0.702924,0.466446,0.263963,0.460945,0.392866,0.461744,0.531243,0.467242,0.661092,0.472422,0.783456,0.497345,0.782750,0.508985,0.657344,0.514091,0.528246,0.507810,0.404656,0.497608,0.272467,cylinder_20
9,73.6,60.5,42.7,64.4,276.9,275.9,0.0,0.0,0.477933,0.168167,0.801732,0.358462,0.797481,0.711192,0.479032,0.878855,0.179511,0.704736,0.466062,0.265051,0.459329,0.394016,0.459408,0.531307,0.465175,0.660174,0.471404,0.782622,0.497041,0.781873,0.508966,0.656590,0.514320,0.528111,0.507990,0.405229,0.497727,0.273183,cylinder_20


In [ ]:
# grasp_dataset.to_csv('/home/dell/Desktop/xxxx/Uni/Kirigami_project/Keypoint_detection_notebooks/Grasp_dataset_3/' + 'validation_df.csv', index=False)

In [46]:
# grasp_data = grasp_dataset.drop(columns=['label', 'weight_reading_1', 'weight_reading_2'], axis=1)
grasp_data = grasp_dataset.drop(columns=['label'], axis=1)
# grasp_data = grasp_dataset.drop(columns=['label', 'p1_x', 'p1_y', 'p2_x', 'p2_y', 'p3_x', 'p3_y', 'p4_x', 'p4_y', 'p5_x', 'p5_y', 'p6_x', 'p6_y', 'p7_x', 'p7_y', 'p8_x', 'p8_y', 'p9_x', 'p9_y', 'p10_x', 'p10_y', 'p11_x', 'p11_y', 'p12_x', 'p12_y', 'p13_x', 'p13_y', 'p14_x', 'p14_y', 'p15_x', 'p15_y', 'p16_x', 'p16_y'], axis=1)
# grasp_data['weight_reading_1'] = grasp_data['weight_reading_1'] / 1000
# grasp_data['weight_reading_2'] = grasp_data['weight_reading_2'] / 1000
grasp_data['pressure_reading_1'] = grasp_data['pressure_reading_1'] / 100
grasp_data['pressure_reading_2'] = grasp_data['pressure_reading_2'] / 100
grasp_data['force_reading_1'] = grasp_data['force_reading_1'] / 1000
grasp_data['force_reading_2'] = grasp_data['force_reading_2'] / 1000
grasp_label = grasp_dataset['label']

print(grasp_data.head(2))
print(grasp_label.head(2))

   weight_reading_1  weight_reading_2  pressure_reading_1  pressure_reading_2  \
0              72.0              61.6               0.394               0.573   
1              74.8              59.3               0.417               0.644   

   force_reading_1  force_reading_2  p1_x  p1_y      p2_x      p2_y      p3_x  \
0           0.2643           0.2637   0.0   0.0  0.474616  0.177147  0.800419   
1           0.2700           0.2693   0.0   0.0  0.476011  0.172187  0.800696   

       p3_y      p4_x      p4_y      p5_x      p5_y      p6_x      p6_y  \
0  0.341526  0.798435  0.722489  0.479104  0.872773  0.184380  0.701780   
1  0.348850  0.797162  0.716827  0.478588  0.875796  0.182801  0.702902   

       p7_x     p7_y      p8_x      p8_y      p9_x      p9_y     p10_x  \
0  0.466461  0.26349  0.463288  0.391953  0.464998  0.531428  0.470341   
1  0.466307  0.26367  0.461439  0.392506  0.462530  0.531179  0.468002   

      p10_y     p11_x     p11_y     p12_x     p12_y     p13_x  

In [42]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn import tree
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier

data_train, data_test, label_train, label_test = train_test_split(grasp_data, grasp_label, test_size=0.2, random_state=np.random.randint(100))

# print(X_train.head(3))

# clf = KNeighborsClassifier(n_neighbors=7)

# knn.fit(X_train, y_train)

# y_pred = knn.predict(X_test)

# accuracy = knn.score(X_test, y_test)
# print('Accuracy:', accuracy)
# clf = RandomForestClassifier(n_estimators=1000, max_depth=50, random_state=np.random.randint(100))
clf = RandomForestClassifier(n_estimators=1000, max_depth=20, random_state=np.random.randint(100), n_jobs = -1)
# clf = HistGradientBoostingClassifier(max_iter=200, random_state=np.random.randint(100))
# clf = GradientBoostingClassifier(n_estimators=200, learning_rate=0.1, max_depth=1, random_state=0)
# clf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)],voting='hard')

# num_folds = 10
# cv_method = KFold(n_splits=num_folds, shuffle=True, random_state=np.random.randint(100))
# cv_results = cross_val_score(clf, grasp_data, grasp_label, cv=cv_method, scoring='accuracy')

# print('Cross-validation results:', cv_results)
# print('Average accuracy:', cv_results.mean())

clf.fit(data_train, label_train)
label_pred = clf.predict(data_test)
cm = confusion_matrix(label_test, label_pred)

# short_dataset_names = ['chicken', 'tomato']


# cm_df = pd.DataFrame(cm, index=short_dataset_names, columns=short_dataset_names)

# print(cm_df)
# print(cm)

accuracy = clf.score(data_test, label_test)
print("Accuracy:", accuracy)

Accuracy: 1.0


In [47]:
label_pred = clf.predict(grasp_data)
print(label_pred)
accuracy = clf.score(grasp_data, grasp_label)
print("Accuracy:", accuracy)

['cylinder_20' 'cylinder_20' 'cylinder_20' 'cylinder_20' 'cylinder_20'
 'cylinder_20' 'cylinder_20' 'cylinder_20' 'cylinder_20' 'cylinder_20'
 'cylinder_40' 'cylinder_40' 'cylinder_40' 'cylinder_40' 'cylinder_40'
 'cylinder_40' 'cylinder_40' 'cylinder_40' 'cylinder_40' 'cylinder_40'
 'cylinder_60' 'cylinder_60' 'cylinder_60' 'cylinder_60' 'cylinder_60'
 'cylinder_60' 'cylinder_60' 'cylinder_60' 'cylinder_60' 'cylinder_60'
 'square_20' 'square_20' 'cylinder_20' 'square_20' 'square_20' 'square_20'
 'square_20' 'square_20' 'square_20' 'square_20' 'square_40' 'square_40'
 'square_40' 'square_40' 'square_40' 'square_40' 'square_40' 'square_40'
 'square_40' 'square_40' 'square_60' 'square_60' 'square_60' 'square_60'
 'square_60' 'square_60' 'square_60' 'square_60' 'square_60' 'square_60']
Accuracy: 0.9833333333333333
